---

# Remote Procedure Calls
### [Emil Sekerinski](http://www.cas.mcmaster.ca/~emil/), McMaster University, February 2022
---

<figure style="float:right;border-right:2em solid white" >
    <img style="width:90pt" src="./img/by-nc-nd.png"/>
    <figcaption style="width:90pt;font-size:x-small"><a href="https://creativecommons.org/licenses/by-nc-nd/4.0/" style="font-size:x-small">Licensed under Creative Commons CC BY-NC-ND</a>
    </figcaption>
</figure>

Message passing is well-suited for one-directional communication, as in producers-consumers, filters, etc. When bidirectional communication between clients and a server is needed, a channel for sending requests and a reply channel for each client needs to be introduced.

The remote procedure call (RPC) abstracts from the channels in client-server communication. The server exports procedures that can be called as if they were local. When a client calls a remote procedure, the arguments of the call are sent to the server and the client is suspended until the results of the remote procedure are sent back to the client.

Assume that the server runs following process:
```
var gcd.args: channel[integer × integer × channel[integer]]

procedure gcd(x, y: integer) → (r: integer)
    do x > y → x := x - y
     ⫿  y > x → y := y - x
    r := x

process server
    do gcd.args ? (x, y, reply) →
            r ← gcd(x, y); reply ! r
```
The server and cliente communicate only by message passing; the do not share any memory. The remote procedure call
```
r ← server.gcd(a, b)
```
in a client is defined as
```
gcd.args ! (a, b, reply) ; reply ? r
```
where `reply` is of type `channel[integer]`. The call has same effect as `r ← gcd(a, b)` if procedure `gcd` were defined locally in the client.

Servers can run multiple procedures; in principle, each procedure `pᵢ` has its own channel `pᵢ.args` over which it receives all the arguments and the reply channel. The server executes first an initialization, `S₀` below, and then in `do` loop accepts messages from any of the channels `pᵢ.args`, after which it executes the corresponding procedure:
```
var pᵢ.args: channel[argsᵢ × channel[resᵢ]]
...
procedure pᵢ(aᵢ: argsᵢ) → (rᵢ: resᵢ)
    Sᵢ
...
process server
    S₀
    do pᵢ.args ? (aᵢ, reply) →  rᵢ ← pᵢ(aᵢ); reply ! rᵢ
    ...
```
If `reply: channel[resᵢ]` is declared by a client, the remote call
```
rᵢ ← server.pᵢ(aᵢ)
```
in the client is defined as:
```
pᵢ.args ! (aᵢ, reply) ; reply ? rᵢ
```
It has same effect as `rᵢ ← pᵢ(aᵢ)` if procedure `pᵢ` were defined locally in the client. However, while the call has the same effect, the remove procedure call will cause a delay and it may fail if communication breaks down.

Since only one of the procedure `pᵢ` executes at any time, the rules for correctness are identical to those for a monitor with initialization `S₀` and procedures `pᵢ`. That is, the correctness of the server is argued as if there were no message passing involved.

With remote procedure calls, the channels are attached to server: the name of the server has to be known to the client. Different programming languages follow different conventions for naming.

### RPC in Python

Following `gcd` server uses the standard [`xmlrpc` library](https://docs.python.org/3/library/xmlrpc.html). The library encodes the parameters and results as XML structures for transmission. The parameter to `SimpleXMLRPCServer` is a tuple with the Internet address and the port number; the port must be opened for communication.

The call to `serve_forever` goes into an infinite loop: before running the cell below, open a copy of this notebook in a separate window with the Jupyter kernel running on the same or a different computer.

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer

def gcd(x,y):
    while x != y:
        if x > y: x -= y
        else: y -= x
    return x

server = SimpleXMLRPCServer(("your.server", 8020)) # use this if running remotely
#server = SimpleXMLRPCServer(("localhost", 8020)) # use this if running locally
server.register_function(gcd, "gcd")
server.serve_forever()

On the client, a _server proxy_ has to be created:

In [ ]:
import xmlrpc.client

#server = xmlrpc.client.ServerProxy("http://your.server:8020") # use this if running remotely
server = xmlrpc.client.ServerProxy("http://localhost:8020") # use this if running locally
server.gcd(81,36)

_Question:_ Suppose there is sequence of calls to `server.gcd`. Do the client and server run in parallel?

With the `gcd` server, either the server or the client would execute, but not both; there could be periods when neither executes due to the time for the transmission.

The `xmlrpc` library also allows objects to be remotely called. The parameter `allow_none=True` is needed when creating the server proxy to allow parameterless calls. (Reminder: open a new copy of the notebook before running next cell)

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer

class Counter:
    def __init__(self):
        self.a, self.e = 0, True
        # e == even(a)
    def inc(self):
        self.a, self.e = self.a + 1, not self.e
    def even(self):
        return self.e

# server = SimpleXMLRPCServer(("your.server", 8020), allow_none=True) # use this if running remotely
server = SimpleXMLRPCServer(("localhost", 8020), allow_none=True) # use this if running locally
server.register_instance(Counter()) # create Counter object, then register
server.serve_forever()

The corresponding client is:

In [ ]:
import xmlrpc.client

# c = xmlrpc.client.ServerProxy("http://your.server:8020") # use this if running remotely
c = xmlrpc.client.ServerProxy("http://localhost:8020") # use this if running locally
c.inc(); c.even()

Trying to run a server on a port that is already in use results in an "address in use" error. To check which ports are currently used, run:

In [ ]:
!netstat -np TCP

To check the status of a specific port, run on Unix-based systems:

In [ ]:
!netstat -np TCP | grep 8020

On Windows, use `findstr` instead of `grep`:

In [ ]:
!netstat -np TCP | findstr "8020"

Alternatively, instead of the default `cmd` shell on Windows, you can use `powershell` with `Select-String`: 

In [ ]:
import os; os.environ['comspec']='powershell.exe'; os.getenv('comspec')

In [ ]:
!netstat -np TCP | Select-String -Pattern 8020

When running a Python RPC server from a notebook, it will only run as long as the notebook runs. To keep a server running after logging out, save the server to a file, say `Counter.py`, and run from the Unix command line (not notebooks):
```
nohup python3 Counter.py &
```
The `&` starts a new process that runs in the background and `nohup` prevents that process from being terminated when logging out. To check the log produced by the server process, run:
```
cat nohup.out
```

While the RPC server corresponds to an active monitor, the server is single-threaded, i.e. only one procedure or method at a time is executed. Python supports also multi-threaded RPC servers by creating a new server class that "mixes in" [ThreadingMixIn](https://docs.python.org/3/library/socketserver.html#socketserver.ThreadingMixIn).

### Classes in Go

In Go, methods of a class do not have to be declared together with the fields. Rather, the fields are declared as a `struct` and methods separately, with the parameter before the method name being the receiver of the call. This allows methods to be added as needed without introducing new classes by inheritance.

In [ ]:
%%writefile point.go
package main
import "math"

type Point struct{X, Y float64}

func (p *Point) Distance() float64 {
    return math.Sqrt(p.X * p.X + p.Y * p.Y)
}
func (p *Point) Scale(factor float64) {
    p.X *= factor; p.Y *= factor
}
func main(){
    q := new(Point); q.X, q.Y = 3, 4
    l := q.Distance(); println(l)
    q.Scale(2); println(q.X, q.Y)
}

In [ ]:
!go run point.go

A class invariant becomes the invariant of a `struct`:

In [ ]:
%%writefile counter.go
package main

type Counter struct{A int32; E bool} // E == even(A)

func (self *Counter) Inc() {
    self.A += 1; self.E = !self.E
}
func (self *Counter) Even() bool {
    return self.E
}
func main(){
    c := new(Counter); c.A, c.E = 0, true
    c.Inc(); println(c.Even())
    c.Inc(); println(c.Even())
}

In [ ]:
!go run counter.go

### RPC in Go

The [net/rpc](https://golang.org/pkg/net/rpc/) package allows remote calls to methods of the form
```Go
func (t *T) MethodName(args ArgsType, reply *ReplyType) error
```
Type `error` is predeclared as
```Go
type error interface {
	Error() string
}
```
By convention, returning `nil` means that no error occurred.

For a GCD server, the function for computing the GCD has to be written as a method. As methods can be attached to (almost) any type, we define a new type `Arith` to be `int` and attach method `Gcd` to `Arith`. The main program first creates an `Arith` object before calling method `Gcd`:

In [ ]:
%%writefile gcdmethod.go
package main

type GcdArgs struct{X, Y int}
type Arith int

func (a *Arith) Gcd(args *GcdArgs, result *int) error {
    x, y := args.X, args.Y
    for x != y {
        if x > y {x -= y} else {y -= x}
    }
    *result = x
    return nil
}
func main() {
    a := new(Arith); println(a); println(*a)
    var r int
    a.Gcd(&GcdArgs{81, 36}, &r)
    println(r)
    b := new(Arith); println(b); println(*b)
}

In [ ]:
!go run gcdmethod.go

_Question:_ What is the output of the `println` statements?

The server registers a new `Arith` object, specifies that HTTP is used for encoding call arguments and results, and then listens to incoming calls. The call to `Serve` goes into an infinite loop: before running the cell below, open a copy of this notebook in a separate window with the Jupyter kernel running on the same or a different computer.

In [ ]:
%%writefile arithserver.go
package main
import ("net"; "net/rpc"; "net/http")

type GcdArgs struct{X, Y int}
type Arith int

func (a *Arith) Gcd(args *GcdArgs, result *int) error {
    x, y := args.X, args.Y
    for x != y {
        if x > y {x -= y} else {y -= x}
    }
    *result = x
    return nil
}
func main(){
    rpc.Register(new(Arith)); rpc.HandleHTTP()
    l, err := net.Listen("tcp", ":8020")
    if err != nil {panic(err)}
    http.Serve(l, nil)
}

In [ ]:
!go run arithserver.go

On the client, first a new object of type `Client` has to be created. That object has a method `Call` that requires the name of the remote RPC object and method to be supplied; the parameters and result values have to be specified as in the server.

Alternatively to calling the remote procedure, it Go allows first send a message to the server with the arguments and the type of the expected result by calling method `Go` of the client object, which also returns a `Call` object. Then the client can wait for the result of the call by receiving from channel `Done` of the call object. In that case, the result of the call is in the field `Reply` of the call object.

In [ ]:
%%writefile arithclient.go
package main
import ("net/rpc")

type GcdArgs struct{X, Y int}

func main() {
    // client, err := rpc.DialHTTP("tcp", "your.server:8020")
    client, err := rpc.DialHTTP("tcp", "localhost:8020")
    if err != nil {panic(err)}
    
    // synchronous call
    var result int
    err = client.Call("Arith.Gcd", &GcdArgs{81, 36}, &result)
    if err != nil {panic(err)}
    println(result)
    
    // asynchronous call
    gcdCall := client.Go("Arith.Gcd", &GcdArgs{10, 4}, &result, nil)
    if gcdCall.Error != nil {panic(err)}
    <-gcdCall.Done //
    if gcdCall.Error != nil {panic(err)}
    println(*gcdCall.Reply.(*int)) // type assertion
}

In [ ]:
!go run arithclient.go

A server for a counter works similarly: a counter object is first created, its fields are initialized, and then the counter object is registered for RPC calls. Since RPC procedures have to have be of previously mentioned form with parameters and results, the empty type `struct{}` (with single value `struct{}{}`) is used where needed: 

In [ ]:
%%writefile counterserver.go
package main
import ("net"; "net/rpc"; "net/http")

type Counter struct{A int32; E bool} // E == even(A)

type Void struct{}

func (self *Counter) Inc(args *Void, result *Void) error {
    self.A += 1; self.E = !self.E; return nil
}
func (self *Counter) Even(args *Void, result *bool) error {
    *result = self.E; return nil
}
func main(){
    c := new(Counter); c.A, c.E = 0, true
    rpc.Register(c); rpc.HandleHTTP()
    l, err := net.Listen("tcp", ":8020")
    if err != nil {panic(err)}
    http.Serve(l, nil)
}

In [ ]:
!go run counterserver.go

In the client code below, auxiliary procedures `inc_counter` and `even_counter` are defined that hide the details of passing the arguments and results:

In [ ]:
%%writefile counterclient.go
package main
import ("net/rpc")

type Void struct{}

func init_counter() *rpc.Client {
    // client, err := rpc.DialHTTP("tcp", "your.server:8020")
    client, err := rpc.DialHTTP("tcp", "localhost:8021")
    if err != nil {panic(err)}
    return client
}
func inc_counter(client *rpc.Client) {
    var result Void
    err := client.Call("Counter.Inc", &Void{}, &result)
    if err != nil {panic(err)}
}
func even_counter(client *rpc.Client) bool {
    var even bool
    err := client.Call("Counter.Even", &Void{}, &even)
    if err != nil {panic(err)}
    return even
}
func main(){
    c := init_counter()
    inc_counter(c); println(even_counter(c))
    inc_counter(c); println(even_counter(c))
}

In [ ]:
!go run counterclient.go